In [1]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Input
from keras import backend as K
from itertools import product

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np
from random import random as rand

In [42]:
def _KL_loss(centers=None, encoded=None):
        if (centers is not None) and (encoded is None):
            def KL_loss(encoded_true, encoded_pred):
                q_arg_array = [encoded_pred, centers]
                q = tf.py_func(_get_q, q_arg_array, tf.float32)
#                 q.set_shape([encoded_pred.get_shape()[0], centers.get_shape()[0]])
                
                f_arg_array = [q]
                f = tf.py_func(_get_f, f_arg_array, tf.float32)
                
                p_arg_array = [q, f]
                p = tf.py_func(_get_p, p_arg_array, tf.float32)
                
                loss = tf.reduce_sum(tf.multiply(p, tf.log(tf.div(p, q))))
                print(K.ndim(loss))
                return loss
            return KL_loss
            
        elif (encoded is not None) and (centers is None):
            def KL_loss(centers_true, centers_pred):
                q_arg_array = [encoded, centers_pred]
                q = tf.py_func(_get_q, q_arg_array, tf.float32)
                q.set_shape([encoded.get_shape()[0], centers_pred.get_shape()[0]])
                
                f_arg_array = [q]
                f = tf.py_func(_get_f, f_arg_array, tf.float32)
                f.set_shape(q.get_shape())
                
                p_arg_array = [q, f]
                p = tf.py_func(_get_p, p_arg_array, tf.float32)
                p.set_shape(q.get_shape())
                
                loss = tf.reduce_sum(tf.multiply(p, tf.log(tf.div(p, q))))
                print(K.ndim(loss))
                return loss
            return KL_loss
            
        else:
            raise Exception("Fuck it!")
            print("Error. You must pass either centers or encoded but not both.")
            return
    
def _get_q(encoded, centers):
#         encoded, centers = arg_array
        cart_products = list(product(enumerate(encoded),
                               enumerate(centers)))
        q = np.ones(shape=(len(encoded), len(centers))) 
        q_normich = np.array([sum([(1 + (z - mu) ** 2) ** (-1)
                                   for (i, z), (j, mu) in cart_products
                                   if i == i_chert]) 
                              for i_chert, _ in enumerate(encoded)])
        for (i, z), (j, mu) in cart_products:
            q[i, j] = (1 + (z - mu) ** 2) ** (-1) / q_normich[i]
        print(np.isnan(q).any())
        q = q.astype(np.float32)
        return q
#     inp = tf.placeholder(tf.float32)
#     _get_q = tf.py_func(_get_q, [inp], tf.float32)
            
def _get_f(q):
#         f = arg_array[0]
        f = np.zeros_like(q)
        for i, _ in enumerate(q):
            f += np.roll(q, i, axis=0)
        f = f.astype(np.float32)
        return f
#     inp = tf.placeholder(tf.float32)
#     _get_f = tf.py_func(_get_f, [inp], tf.float32)
    
def _get_p(q, f):
#         q, f = arg_array
        # get p_normich, hoole
        p_normich_orig = q ** 2 / f
        p_normich = np.zeros_like(p_normich_orig)
        for j in range(p_normich_orig.shape[1]):
            p_normich += np.roll(p_normich_orig, j, axis=1)

        p = q ** 2 / f / p_normich
        p = p.astype(np.float32)
        return p
#     inp = tf.placeholder(tf.float32)
#     _get_p = tf.py_func(_get_p, [inp], tf.float32)

## Generate simulation data

In [ ]:
emb_dim = 2

In [ ]:
train_data

## Test fuckin' model

In [ ]:
with tf.Session() as sess:
    input_data_layer = Input(shape=train_data[0].shape) 
    input_ones_layer = Input(shape=train_ones[0].shape)
    
    pointwise = LocallyConnected2D(1, kernel_size=(1, 1), use_bias=False)
    pointwise_called = pointwise(input_ones_layer)
    pointwise.set_weights(np.reshape(centers, (1, np.product(centers.shape), 1, 1)))
    
    last = Dense(emb_dim, activation="sigmoid")(input_data_layer)
            
    model = Model(inputs=[input_data_layer, input_ones_layer], outputs=[last, pointwise_called])
    model.compile(optimizer=self.optimizer, 
                  loss=[self._KL_loss(centers=pointwise_called), self._KL_loss(encoded=last)])

In [38]:
def _get_f(q):
    f_unstacked = tf.reduce_sum(q, axis=0)
    f = tf.stack([f_unstacked for _ in range(batch_size)])
    return f

In [40]:
def _get_p(q, f):
    numerator = tf.divide(tf.multiply(q, q), f)
    almost_denominator = tf.reduce_sum(numerator, axis=1)
    denominator = tf.stack([almost_denominator 
                            for _ in range(n_clusters)], axis=1)
    p = tf.divide(numerator, denominator)
    #------------------------------------------
    return p

In [49]:
with tf.Session() as sess:
    batch_size = 6
    n_clusters = 2
    centers = tf.constant([[1., 2.], [3., 4.]])
    centers_stacked = tf.stack([centers for _ in range(batch_size)], axis=0)
    
    embedded = tf.constant([[1., 2.], [3., 3.], [1., 5.], [1., 5.], [3., 4.], [1., 2.]])
    embedded_stacked = tf.stack([embedded for _ in range(n_clusters)], axis=1)
    
    q = tf.subtract(embedded_stacked, centers_stacked)
    q_sqr = tf.multiply(q, q)
    q_normed_sqr = tf.reduce_sum(q_sqr, axis=2)
    ones = tf.ones_like(q_normed_sqr)
    
    numerator = tf.divide(ones, tf.add(ones, q_normed_sqr))
    
    almost_denominator = tf.reduce_sum(numerator, axis=1)
    denominator = tf.stack([almost_denominator for _ in range(2)], axis=1)
    
    true_q = tf.divide(numerator, denominator)
    
#     keks_normed = tf.norm(keks, ord="euclidean", axis=2)
#     keks_normed_sqr = tf.multiply(keks_normed, keks_normed)

    f = _get_f(true_q)
    p = _get_p(true_q, f)
    loss = tf.reduce_sum(tf.multiply(p, tf.log(tf.divide(p, true_q))))
    
    print(centers_stacked.eval(), end="\n\n\n")
    print(embedded_stacked.eval(), end="\n\n\n")

    print(q.eval(), end="\n\n\n")
    print(q_normed_sqr.eval(), end="\n\n\n")
    print(numerator.eval(), end="\n\n\n")
    print(denominator.eval(), end="\n\n\n")
    print("q is", true_q.eval(), end="\n\n\n")
    print("f is", f.eval(), end="\n\n\n")
    print("p is", p.eval(), end="\n\n\n")
    print(loss.eval(), end="\n\n\n")
    
#     print(keks.eval().shape)

[[[1. 2.]
  [3. 4.]]

 [[1. 2.]
  [3. 4.]]

 [[1. 2.]
  [3. 4.]]

 [[1. 2.]
  [3. 4.]]

 [[1. 2.]
  [3. 4.]]

 [[1. 2.]
  [3. 4.]]]


[[[1. 2.]
  [1. 2.]]

 [[3. 3.]
  [3. 3.]]

 [[1. 5.]
  [1. 5.]]

 [[1. 5.]
  [1. 5.]]

 [[3. 4.]
  [3. 4.]]

 [[1. 2.]
  [1. 2.]]]


[[[ 0.  0.]
  [-2. -2.]]

 [[ 2.  1.]
  [ 0. -1.]]

 [[ 0.  3.]
  [-2.  1.]]

 [[ 0.  3.]
  [-2.  1.]]

 [[ 2.  2.]
  [ 0.  0.]]

 [[ 0.  0.]
  [-2. -2.]]]


[[0. 8.]
 [5. 1.]
 [9. 5.]
 [9. 5.]
 [8. 0.]
 [0. 8.]]


[[1.         0.11111111]
 [0.16666667 0.5       ]
 [0.1        0.16666667]
 [0.1        0.16666667]
 [0.11111111 1.        ]
 [1.         0.11111111]]


[[1.1111112  1.1111112 ]
 [0.6666667  0.6666667 ]
 [0.26666668 0.26666668]
 [0.26666668 0.26666668]
 [1.1111112  1.1111112 ]
 [1.1111112  1.1111112 ]]


q is [[0.9        0.09999999]
 [0.25       0.75      ]
 [0.375      0.625     ]
 [0.375      0.625     ]
 [0.09999999 0.9       ]
 [0.9        0.09999999]]


f is [[2.9 3.1]
 [2.9 3.1]
 [2.9 3.1]
 [2.9 3.1]
 [2.

In [5]:
np.reshape(table, (3, 2))

array([[1, 2],
       [3, 3],
       [4, 7]])